In [9]:
from google.colab import files

uploaded = files.upload()

Saving vaccination_tweets.csv to vaccination_tweets.csv


In [3]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import re
import itertools
from wordcloud import WordCloud, STOPWORDS
from collections import Counter
import nltk
nltk.download('vader_lexicon')
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
import tweepy as tw
from collections import Counter

ModuleNotFoundError: No module named 'wordcloud'

In [102]:
import tweepy as tw
import pandas as pd

auth = tw.OAuthHandler('d5HZe4mtTtZDAJllEnbuyaT9v', 'mY7Jlf9r1j3BOI20QSB84T3ktByMxjwjAhL6rrRRjrBckBShE0')
auth.set_access_token('230756541-J0P9NtPULFfEebC5mOQoB1k4n6dJwd7yk8NPGjHd', 'DY0UlIsKTG6FX7AuQVUCTQx248Mbta8J76FuSyHTMezDB')
api = tw.API(auth)
search_words = "#PfizerVaccine OR #Vaccine OR #Moderna OR #pfizervaccine OR #vaccination"
date_since = "2021-01-01"
date_until= "2021-01-10"

tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(1000)
content = []
id = []
username = []
location = []
description = []
user_creat_date = []
followers = []
favorite_tweet_count = []
retweet_count = []
posted_at = []
is_retweet = []
for tweet in tweets:
    content.append(tweet.text)
    id.append(tweet.id)
    username.append(tweet.user.screen_name)
    location.append(tweet.user.location)
    description.append(tweet.user.description)
    user_creat_date.append(tweet.user.created_at)
    followers.append(tweet.user.followers_count)
    favorite_tweet_count.append(tweet.favorite_count)
    retweet_count.append(tweet.retweet_count)
    posted_at.append(tweet.created_at)
    is_retweet.append(tweet.retweet_count > 0)


data = {'id': id,
        'user_name': username,
        'user_location':location,
        'user_description':description,
        'user_created': user_creat_date,
        'user_followers':followers,
        'date':posted_at,
        'text':content,
        'favorites':favorite_tweet_count,
        'retweets':retweet_count,
        'is_retweet':is_retweet
        }

# Create DataFrame
#

df1 = pd.DataFrame(data)


In [103]:
df1.head()

,id,user_name,user_location,user_description,user_created,user_followers,date,text,favorites,retweets,is_retweet
0,1352199721834655745,buzz_bmth,"Bournemouth, UK",BUzz is an online news service from @bournemou...,2009-03-10 09:12:15,1500,2021-01-21 10:21:50,1) Dr Zahid Chauhan is the man behind Oldham C...,0,0,False
1,1352199618692608001,PatKennyNT,Ireland,Every weekday from 9am - 12pm on @NewstalkFM. ...,2013-08-02 12:32:17,64223,2021-01-21 10:21:26,Coming up we ask the vaccine deal an agreeable...,0,0,False
2,1352199473838100481,PhilACarpenter,Drawbridge Britain,alfisti | ai | bianchi | cdp | cx | cyclist | ...,2009-02-18 22:06:27,2274,2021-01-21 10:20:51,RT @SkyNewsThomas: Chief scientist says UK wil...,0,853,True
3,1352199471241830402,paullantz,"Belleville, Ontario",Retired legal aid clinic lawyer and former com...,2010-09-15 20:09:20,3807,2021-01-21 10:20:51,"RT @weeneebayko: Anna Belle, 96 years old and ...",0,12,True
4,1352199422864748544,PeterHotez,"Montrose, Houston TX","Vaccine Scientist-Pediatrician-Author, Prof De...",2012-05-29 00:46:55,124717,2021-01-21 10:20:39,RT @saribethrose: Thank you @Reuters for follo...,0,24,True


In [104]:
df1['hashtags']=df1['text'].apply(lambda x: ", ".join(re.findall('(#+[a-zA-Z0-9(_)]{1,})',x)))

In [105]:
df = pd.read_csv('vaccination_tweets.csv')

In [106]:
df.head()

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet
0,1340539111971516416,Rachel Roh,"La Crescenta-Montrose, CA",Aggregator of Asian American news; scanning di...,2009-04-08 17:52:46,405,1692,3247,False,2020-12-20 06:06:44,Same folks said daikon paste could treat a cyt...,['PfizerBioNTech'],Twitter for Android,0,0,False
1,1338158543359250433,Albert Fong,"San Francisco, CA","Marketing dude, tech geek, heavy metal & '80s ...",2009-09-21 15:27:30,834,666,178,False,2020-12-13 16:27:13,While the world has been on the wrong side of ...,NaN,Twitter Web App,1,1,False
2,1337858199140118533,eli🇱🇹🇪🇺👌,Your Bed,"heil, hydra 🖐☺",2020-06-25 23:30:28,10,88,155,False,2020-12-12 20:33:45,#coronavirus #SputnikV #AstraZeneca #PfizerBio...,"['coronavirus', 'SputnikV', 'AstraZeneca', 'Pf...",Twitter for Android,0,0,False
3,1337855739918835717,Charles Adler,"Vancouver, BC - Canada","Hosting ""CharlesAdlerTonight"" Global News Radi...",2008-09-10 11:28:53,49165,3933,21853,True,2020-12-12 20:23:59,"Facts are immutable, Senator, even when you're...",NaN,Twitter Web App,446,2129,False
4,1337854064604966912,Citizen News Channel,NaN,Citizen News Channel bringing you an alternati...,2020-04-23 17:58:42,152,580,1473,False,2020-12-12 20:17:19,Explain to me again why we need a vaccine @Bor...,"['whereareallthesickpeople', 'PfizerBioNTech']",Twitter for iPhone,0,0,False


In [107]:
df = pd.concat([df,df1], ignore_index=True)

In [108]:
df

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet
0,1340539111971516416,Rachel Roh,"La Crescenta-Montrose, CA",Aggregator of Asian American news; scanning di...,2009-04-08 17:52:46,405,1692.0,3247.0,False,2020-12-20 06:06:44,Same folks said daikon paste could treat a cyt...,['PfizerBioNTech'],Twitter for Android,0,0,False
1,1338158543359250433,Albert Fong,"San Francisco, CA","Marketing dude, tech geek, heavy metal & '80s ...",2009-09-21 15:27:30,834,666.0,178.0,False,2020-12-13 16:27:13,While the world has been on the wrong side of ...,NaN,Twitter Web App,1,1,False
2,1337858199140118533,eli🇱🇹🇪🇺👌,Your Bed,"heil, hydra 🖐☺",2020-06-25 23:30:28,10,88.0,155.0,False,2020-12-12 20:33:45,#coronavirus #SputnikV #AstraZeneca #PfizerBio...,"['coronavirus', 'SputnikV', 'AstraZeneca', 'Pf...",Twitter for Android,0,0,False
3,1337855739918835717,Charles Adler,"Vancouver, BC - Canada","Hosting ""CharlesAdlerTonight"" Global News Radi...",2008-09-10 11:28:53,49165,3933.0,21853.0,True,2020-12-12 20:23:59,"Facts are immutable, Senator, even when you're...",NaN,Twitter Web App,446,2129,False
4,1337854064604966912,Citizen News Channel,NaN,Citizen News Channel bringing you an alternati...,2020-04-23 17:58:42,152,580.0,1473.0,False,2020-12-12 20:17:19,Explain to me again why we need a vaccine @Bor...,"['whereareallthesickpeople', 'PfizerBioNTech']",Twitter for iPhone,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5042,1352164095408148485,AngelaWLangfor1,,I just don't feel like I have to explain myself.,2020-10-19 05:37:02,0,NaN,NaN,NaN,2021-01-21 08:00:16,RT @TapShort: COVID-19 NEWS SUMMARY\nMORNING N...,"#covid19test, #covi, #newcovid, #covid_19, #ne...",NaN,20,0,True
5043,1352164075325829121,cartoonmovement,Amsterdam,An award-winning collaborative publishing plat...,2010-09-23 08:48:31,27594,NaN,NaN,NaN,2021-01-21 08:00:12,"This week, the WHO said the world is on the br...",,NaN,7,15,True
5044,1352164071718621184,ZioNaziOdor,,I was called a unusual goy from a chosen racer.,2020-11-04 13:35:35,13,NaN,NaN,NaN,2021-01-21 08:00:11,RT @SilenceNotGold: Yet another #vaccination d...,"#vaccination, #Gaza, #Palestine, #DPT",NaN,1,0,True
5045,1352164070984617985,TandrimaChakra1,India,A marketer | A travel blogger | A passionate r...,2017-03-04 05:53:22,29,NaN,NaN,NaN,2021-01-21 08:00:11,"As per Union Health Ministry, a total of 7,86,...",,NaN,0,0,False


convert to datetime type

In [109]:
df['user_created']=pd.to_datetime(df['user_created'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
df['date']=pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

fill na

In [110]:
df['user_description'] = df['user_description'].fillna('No Description')
df['hashtags'] = df['hashtags'].fillna('')
df['source'] = df['source'].fillna('No Source')

NLP preprocessing

add column with clean tweet

In [111]:
df["up_text"] = df.text.str.lower()
#Remove twitter handlers
df.up_text = df.up_text.apply(lambda x:re.sub('@[^\s]+','',x))

#remove hashtags
df.up_text = df.up_text.apply(lambda x:re.sub(r'\B#\S+','',x))

# Remove URLS
df.up_text = df.up_text.apply(lambda x:re.sub(r"http\S+", "", x))

# Remove all the special characters
df.up_text = df.up_text.apply(lambda x:' '.join(re.findall(r'\w+', x)))

#remove all single characters
df.up_text = df.up_text.apply(lambda x:re.sub(r'\s+[a-zA-Z]\s+', '', x))

# Substituting multiple spaces with single space
df.up_text = df.up_text.apply(lambda x:re.sub(r'\s+', ' ', x, flags=re.I))

df.sort_values("date", inplace=True)


In [112]:
# Contractions
df.up_text = df.up_text.apply(lambda x:re.sub(r"he's", "he is", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"there's", "there is", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"We're", "We are", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"That's", "That is", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"won't", "will not", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"they're", "they are", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"Can't", "Cannot", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"wasn't", "was not", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"don\x89Ûªt", "do not", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"aren't", "are not", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"isn't", "is not", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"What's", "What is", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"haven't", "have not", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"hasn't", "has not", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"There's", "There is", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"He's", "He is", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"It's", "It is", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"You're", "You are", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"I'M", "I am", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"shouldn't", "should not", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"wouldn't", "would not", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"i'm", "I am", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"I\x89Ûªm", "I am", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"I'm", "I am", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"Isn't", "is not", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"Here's", "Here is", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"you've", "you have", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"you\x89Ûªve", "you have", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"we're", "we are", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"what's", "what is", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"couldn't", "could not", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"we've", "we have", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"it\x89Ûªs", "it is", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"doesn\x89Ûªt", "does not", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"It\x89Ûªs", "It is", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"Here\x89Ûªs", "Here is", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"who's", "who is", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"I\x89Ûªve", "I have", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"y'all", "you all", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"can\x89Ûªt", "cannot", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"would've", "would have", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"it'll", "it will", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"we'll", "we will", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"wouldn\x89Ûªt", "would not", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"We've", "We have", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"he'll", "he will", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"Y'all", "You all", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"Weren't", "Were not", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"Didn't", "Did not", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"they'll", "they will", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"they'd", "they would", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"DON'T", "DO NOT", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"That\x89Ûªs", "That is", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"they've", "they have", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"i'd", "I would", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"should've", "should have", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"You\x89Ûªre", "You are", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"where's", "where is", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"Don\x89Ûªt", "Do not", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"we'd", "we would", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"i'll", "I will", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"weren't", "were not", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"They're", "They are", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"Can\x89Ûªt", "Cannot", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"you\x89Ûªll", "you will", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"I\x89Ûªd", "I would", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"let's", "let us", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"it's", "it is", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"can't", "cannot", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"don't", "do not", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"you're", "you are", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"i've", "I have", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"that's", "that is", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"i'll", "I will", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"doesn't", "does not", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"i'd", "I would", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"didn't", "did not", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"ain't", "am not", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"you'll", "you will", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"I've", "I have", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"Don't", "do not", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"I'll", "I will", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"I'd", "I would", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"Let's", "Let us", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"you'd", "You would", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"Ain't", "am not", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"Haven't", "Have not", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"Could've", "Could have", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"youve", "you have", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"donå«t", "do not", x))
# Character entity references
df.up_text = df.up_text.apply(lambda x:re.sub(r"&gt;", ">", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"&lt;", "<", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"&amp;", "&", x))
# Typos, slang and informal abbreviations
df.up_text = df.up_text.apply(lambda x:re.sub(r"w/e", "whatever", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"w/", "with", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"USAgov", "USA government", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"recentlu", "recently", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"Ph0tos", "Photos", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"amirite", "am I right", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"exp0sed", "exposed", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"<3", "love", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"amageddon", "armageddon", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"Trfc", "Traffic", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"WindStorm", "Wind Storm", x))
df.up_text = df.up_text.apply(lambda x:re.sub(r"lmao", "laughing my ass off", x))

Clean hashtag column

In [113]:
df.hashtags=df.hashtags.str.replace('[','')
df.hashtags=df.hashtags.str.replace(']','')
df.hashtags=df.hashtags.str.replace("'",'')
df.hashtags=df.hashtags.str.replace("#",'')

if the tweet is only #, up_text = hashtag list

In [114]:
df.loc[df["up_text"] == '','up_text'] = df["hashtags"]

remove tweet that are only @

In [115]:
df.drop(index=df[df['up_text']==""].index, inplace=True)

In [116]:
df.head()

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet,up_text
44,1337727767551553536,Daily News Egypt,Egypt,Egypt's Only Daily Independent Newspaper in En...,2009-04-26 07:56:24,278080,116.0,765.0,True,2020-12-12 11:55:28,#FDA authorizes #PfizerBioNTech #coronavirus v...,"FDA, PfizerBioNTech, coronavirus",Twitter Web App,1,1,False,authorizes vaccine for emergency use
43,1337728701820268544,City A.M.,"London, England","London's business newspaper - News, Opinion, a...",2009-06-09 13:53:06,66224,603.0,771.0,True,2020-12-12 11:59:10,Trump announces #vaccine rollout 'in less than...,vaccine,Twitter Web App,0,2,False,trump announces rollout in less than 24 hours ...
42,1337732077278662657,mystic,NaN,I believe in the goodness of humanity.,2020-05-20 10:57:00,14,162.0,1341.0,False,2020-12-12 12:12:35,#CovidVaccine #Pfizer-BioNTech COVID-19 #Pfize...,"CovidVaccine, Pfizer, PfizerBioNTech, COVID19,...",Twitter for iPhone,0,1,False,covid 19
41,1337732996380696578,KHuntson,USA🇺🇸,"RN, health educator, writer, researcher, wife,...",2014-01-31 01:25:24,2035,1672.0,6734.0,False,2020-12-12 12:16:14,FACT SHEET FOR RECIPIENTS AND CAREGIVERS\n\n#P...,"PfizerBioNTech, CovidVaccine",Twitter Web App,0,0,False,fact sheet for recipients and caregivers
40,1337733048914358272,"Party politics isn't working for anyone, anywhere","Wales, United Kingdom",No lie to another is respectful; no lie to you...,2016-08-03 20:11:00,761,504.0,33675.0,False,2020-12-12 12:16:27,@mirekR9 @sputnikvaccine @AstraZeneca it's the...,PfizerBioNTech,Twitter Web App,0,0,False,itthe only one which doesnhave any side effect...


Sentiment analysis

In [117]:
sid = SIA()
df['sia'] = df['up_text'].apply(lambda x: sid.polarity_scores(' '.join(re.findall(r'\w+',x.lower()))))
df['positive sentiments'] = df['sia'].apply(lambda x: x['pos']+1*(10**-6)) 
df['neutral sentiments'] = df['sia'].apply(lambda x: x['neu']+1*(10**-6))
df['negative sentiments'] = df['sia'].apply(lambda x: x['neg']+1*(10**-6))

df.drop(columns=['sia'],inplace=True)

In [132]:
def get_sentiment(df):
  sent = [df['negative sentiments'],df['positive sentiments']]
  max = np.argmax(sent)
  if df['negative sentiments']== df['positive sentiments']:
    return "Neut"
  elif max ==0:
    return 'Neg'
  else:
    return 'Pos'

Define tweet sentiment

In [133]:
df['sentiments'] =df.apply(get_sentiment,axis=1)

add lemma column

In [139]:
def tokenization(text):
    text = re.split('\W+', text)
    return text

df['tokenized'] = df['up_text'].apply(lambda x: tokenization(x.lower()))
stopword = stopwords.words('english')
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text
    
df['No_stopwords'] = df['tokenized'].apply(lambda x: remove_stopwords(x))

In [140]:
wn = nltk.WordNetLemmatizer()

def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text]
    return text

df['lemmatized'] = df['No_stopwords'].apply(lambda x: lemmatizer(x))

In [141]:
df['lemmatized_dict']=df['lemmatized'].apply(lambda x: Counter(x))

remove unnecessery columns

In [142]:
df.head()

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet,up_text,positive sentiments,neutral sentiments,negative sentiments,sentiments,tokenized,No_stopwords,lemmatized,lemmatized_dict
44,1337727767551553536,Daily News Egypt,Egypt,Egypt's Only Daily Independent Newspaper in En...,2009-04-26 07:56:24,278080,116.0,765.0,True,2020-12-12 11:55:28,#FDA authorizes #PfizerBioNTech #coronavirus v...,"FDA, PfizerBioNTech, coronavirus",Twitter Web App,1,1,False,authorizes vaccine for emergency use,0.000001,0.606001,0.394001,Neg,"[authorizes, vaccine, for, emergency, use]","[authorizes, vaccine, emergency, use]","[authorizes, vaccine, emergency, use]","{'authorizes': 1, 'vaccine': 1, 'emergency': 1..."
43,1337728701820268544,City A.M.,"London, England","London's business newspaper - News, Opinion, a...",2009-06-09 13:53:06,66224,603.0,771.0,True,2020-12-12 11:59:10,Trump announces #vaccine rollout 'in less than...,vaccine,Twitter Web App,0,2,False,trump announces rollout in less than 24 hours ...,0.000001,1.000001,0.000001,Neut,"[trump, announces, rollout, in, less, than, 24...","[trump, announces, rollout, less, 24, hours, f...","[trump, announces, rollout, le, 24, hour, firs...","{'trump': 1, 'announces': 1, 'rollout': 1, 'le..."
42,1337732077278662657,mystic,NaN,I believe in the goodness of humanity.,2020-05-20 10:57:00,14,162.0,1341.0,False,2020-12-12 12:12:35,#CovidVaccine #Pfizer-BioNTech COVID-19 #Pfize...,"CovidVaccine, Pfizer, PfizerBioNTech, COVID19,...",Twitter for iPhone,0,1,False,covid 19,0.000001,1.000001,0.000001,Neut,"[covid, 19]","[covid, 19]","[covid, 19]","{'covid': 1, '19': 1}"
41,1337732996380696578,KHuntson,USA🇺🇸,"RN, health educator, writer, researcher, wife,...",2014-01-31 01:25:24,2035,1672.0,6734.0,False,2020-12-12 12:16:14,FACT SHEET FOR RECIPIENTS AND CAREGIVERS\n\n#P...,"PfizerBioNTech, CovidVaccine",Twitter Web App,0,0,False,fact sheet for recipients and caregivers,0.000001,1.000001,0.000001,Neut,"[fact, sheet, for, recipients, and, caregivers]","[fact, sheet, recipients, caregivers]","[fact, sheet, recipient, caregiver]","{'fact': 1, 'sheet': 1, 'recipient': 1, 'careg..."
40,1337733048914358272,"Party politics isn't working for anyone, anywhere","Wales, United Kingdom",No lie to another is respectful; no lie to you...,2016-08-03 20:11:00,761,504.0,33675.0,False,2020-12-12 12:16:27,@mirekR9 @sputnikvaccine @AstraZeneca it's the...,PfizerBioNTech,Twitter Web App,0,0,False,itthe only one which doesnhave any side effect...,0.000001,1.000001,0.000001,Neut,"[itthe, only, one, which, doesnhave, any, side...","[itthe, one, doesnhave, side, effects]","[itthe, one, doesnhave, side, effect]","{'itthe': 1, 'one': 1, 'doesnhave': 1, 'side':..."


In [143]:
df.columns

Index(['id', 'user_name', 'user_location', 'user_description', 'user_created',
       'user_followers', 'user_friends', 'user_favourites', 'user_verified',
       'date', 'text', 'hashtags', 'source', 'retweets', 'favorites',
       'is_retweet', 'up_text', 'positive sentiments', 'neutral sentiments',
       'negative sentiments', 'sentiments', 'tokenized', 'No_stopwords',
       'lemmatized', 'lemmatized_dict'],
      dtype='object')

In [144]:
df.drop(['tokenized', 'No_stopwords','lemmatized'], axis=1, inplace=True)

Save df to csv

In [145]:
df.to_csv('/content/gdrive/MyDrive/tweet_up_1235.csv')